In [ ]:
import torch
import torch.nn as nn

import clip
import numpy as np


### 1. CLIP

In [ ]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

In [ ]:
model, preprocess = clip.load("ViT-B/32")
input_resolution = model.visual.input_resolution

print("Input resolution:", input_resolution)

In [ ]:
trans = model.transformer
trans

### 2. Data

#### 2.0.0 CSI $\rightarrow$ CSI-Ratio

In [ ]:
import numpy as np
import math

# 假设CSI数据的形状为 (num_packets, num_antennas, num_subcarriers)
# num_packets: 数据包数量（时间序列）
# num_antennas: 天线数量（3根天线）
# num_subcarriers: 子载波数量（30个子载波）

def calculate_amplitude_variance_ratio(csi_data):
    """
    计算每个天线的所有子载波的幅度方差比之和
    :param csi_data: CSI数据，形状为 (num_packets, num_antennas, num_subcarriers)
    :return: 每个天线的幅度方差比之和，形状为 (num_antennas,)
    """
    # 计算幅度
    amplitude = np.abs(csi_data)  # 形状为 (num_packets, num_antennas, num_subcarriers)

    # 计算每个天线和子载波的幅度方差
    variance = np.var(amplitude, axis=0)  # 形状为 (num_antennas, num_subcarriers)

    # 计算每个天线的幅度方差比之和
    variance_ratio_sum = np.sum(variance, axis=1)  # 形状为 (num_antennas,)

    return variance_ratio_sum

def select_reference_antenna(csi_data):
    """
    选择参考天线（幅度方差比之和最小的天线）
    :param csi_data: CSI数据，形状为 (num_packets, num_antennas, num_subcarriers)
    :return: 参考天线的索引
    """
    variance_ratio_sum = calculate_amplitude_variance_ratio(csi_data)
    reference_antenna_index = np.argmin(variance_ratio_sum)
    return reference_antenna_index

def calculate_csi_ratio(csi_data):
    """
    计算CSI比值（CSI-Ratio）
    :param csi_data: CSI数据，形状为 (num_packets, num_antennas, num_subcarriers)
    :return: CSI比值数据，形状为 (num_packets, num_antennas, num_subcarriers)
    """
    num_packets, num_antennas, num_subcarriers = csi_data.shape

    # 选择参考天线
    reference_antenna_index = select_reference_antenna(csi_data)
    reference_csi = csi_data[:, reference_antenna_index, :]  # 参考天线的CSI数据

    # 计算CSI比值
    csi_ratio = np.zeros_like(csi_data)
    for antenna in range(num_antennas):
        if antenna == reference_antenna_index:
            # 参考天线的CSI比值为1
            csi_ratio[:, antenna, :] = 1.0
        else:
            # 其他天线的CSI比值
            csi_ratio[:, antenna, :] = csi_data[:, antenna, :] / reference_csi

    return csi_ratio

In [ ]:
# 示例：生成随机CSI数据
num_packets = 100  # 假设有100个数据包
num_antennas = 3   # 3根天线
num_subcarriers = 30  # 30个子载波
csi_data = np.random.rand(num_packets, num_antennas, num_subcarriers) + 1j * np.random.rand(num_packets, num_antennas, num_subcarriers)

# 计算CSI比值
csi_ratio = calculate_csi_ratio(csi_data)

# 输出结果
print("CSI比值数据的形状:", csi_ratio.shape)

#### 2.0.1 CSI Resample

In [ ]:
import numpy as np
from scipy.interpolate import interp1d

def resample_csi_sequence(csi_sequence, target_length=500):
    """
    对单个CSI序列进行重采样至目标长度
    :param csi_sequence: 原始CSI序列，形状为 (original_length, num_antennas, num_subcarriers)
    :param target_length: 目标序列长度（默认为500）
    :return: 重采样后的CSI序列，形状为 (target_length, num_antennas, num_subcarriers)
    """
    original_length = csi_sequence.shape[0]
    num_antennas = csi_sequence.shape[1]
    num_subcarriers = csi_sequence.shape[2]
    
    # 创建新时间轴
    original_time = np.linspace(0, 1, original_length)  # 归一化时间轴
    new_time = np.linspace(0, 1, target_length)          # 目标时间轴
    
    # 初始化输出序列
    resampled_sequence = np.zeros((target_length, num_antennas, num_subcarriers), dtype=np.complex_)
    
    # 对每个天线和子载波进行插值
    for ant in range(num_antennas):
        for sc in range(num_subcarriers):
            # 提取原始复数值序列
            complex_sequence = csi_sequence[:, ant, sc]
            
            # 创建插值函数（对实部和虚部分别插值）
            interp_real = interp1d(original_time, complex_sequence.real, kind='linear')
            interp_imag = interp1d(original_time, complex_sequence.imag, kind='linear')
            
            # 生成新序列
            resampled_real = interp_real(new_time)
            resampled_imag = interp_imag(new_time)
            
            # 组合复数值
            resampled_sequence[:, ant, sc] = resampled_real + 1j * resampled_imag
            
    return resampled_sequence

def batch_resample(csi_data, target_length=500):
    """
    批量处理CSI数据集
    :param csi_data: 原始CSI数据集，形状为 (num_samples, original_length, num_antennas, num_subcarriers)
    :param target_length: 目标序列长度
    :return: 重采样后的数据集，形状为 (num_samples, target_length, num_antennas, num_subcarriers)
    """
    num_samples = csi_data.shape[0]
    resampled_data = []
    
    for i in range(num_samples):
        resampled_seq = resample_csi_sequence(csi_data[i], target_length)
        resampled_data.append(resampled_seq)
    
    return np.array(resampled_data)

In [ ]:
# 生成示例数据
np.random.seed(0)

# 创建不同长度的CSI序列
raw_data = [
    np.random.randn(300, 3, 30) + 1j*np.random.randn(300, 3, 30),  # 长度300
    np.random.randn(750, 3, 30) + 1j*np.random.randn(750, 3, 30),  # 长度750
    np.random.randn(480, 3, 30) + 1j*np.random.randn(480, 3, 30)   # 长度480
]

# 执行重采样
resampled_data = batch_resample(np.array(raw_data, dtype=object), 500)

# 验证输出形状
print("原始序列长度:", [seq.shape[0] for seq in raw_data])
print("重采样后长度:", [seq.shape[0] for seq in resampled_data])
print("示例数据形状（第一个样本）:", resampled_data[0].shape)

#### 2.1 CSI Dataset 

In [ ]:
import scipy.io as sio
import os
import numpy as np
from typing import List
import random
import pywt

from dataset.data import get_csi_data
from dataset.datasets import CSI_Dataset, HAR_Dataset
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader

In [ ]:
# Widar
data_path = '../data'
batch_size = 20
time_length = 2000
req_domains = ['users', 'gesture_type', 'tosor_loc', 'face_ori', 'rec_dev']

In [ ]:
train_datas, train_labels, eval_datas, eval_labels = get_csi_data(
    data_path, 
    req_domains=req_domains,
)

train_labels.shape

In [ ]:
# HAR
train_datas = HAR_Dataset('D:/NTU-Fi_HAR/train_amp', time_length=2000)

#### 2.2 DFS Data

In [ ]:
import scipy.io as sio
import os
import numpy as np
from typing import List
import random
import pywt

from dataset.data import get_dfs_data
from dataset.datasets import DFS_Dataset
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader

In [ ]:
dfs_data_path = '../dfs_data'
req_domains = ['users', 'gesture_type', 'tosor_loc', 'face_ori']

In [ ]:
train_datas, train_labels, eval_datas, eval_labels = get_dfs_data(dfs_data_path, req_domains)

In [ ]:
train_dataset = DFS_Dataset(
    dfs_data_path,
    train_datas,
    train_labels
)

### 3.Model

In [ ]:
from model.embed import TokenEmbedding

from einops import rearrange
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary


### GPT2

In [1]:
import torch
from transformers.models.gpt2.modeling_gpt2 import GPT2Model, GPT2Config

d:\Anaconda\envs\Deep_Learning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model_config = GPT2Config.from_pretrained('openai-community/gpt2')
model_config

In [3]:
# GPT2 Model
model = GPT2Model.from_pretrained('openai-community/gpt2')
model

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2SdpaAttention(
        (c_attn): Conv1D(nf=2304, nx=768)
        (c_proj): Conv1D(nf=768, nx=768)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D(nf=3072, nx=768)
        (c_proj): Conv1D(nf=768, nx=3072)
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

In [4]:
word_weight = model.get_input_embeddings().weight
word_weight.requires_grad

True

### Qwen2.5-3B

In [ ]:
import torch
from transformers import Qwen2ForCausalLM, Qwen2Model, Qwen2Config

from peft import get_peft_model, LoraConfig, TaskType

In [ ]:
model_config = Qwen2Config.from_pretrained('unsloth/Qwen2.5-3B')
model_config.num_hidden_layers = 12
model_config

In [ ]:
model = Qwen2Model.from_pretrained('unsloth/Qwen2.5-3B', config=model_config)
model

#### Llama-3-8B

In [5]:
import torch
from transformers import LlamaModel, LlamaConfig

In [6]:
model_config = LlamaConfig.from_pretrained('unsloth/Llama-3.2-1B')
model_config

LlamaConfig {
  "_name_or_path": "meta-llama/Llama-3.2-1B",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pad_token_id": 128004,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.46.2",
  "unsloth_fixed": true,
  "use_cache": true,
  "vocab_size": 128256
}

In [ ]:
model = LlamaModel.from_pretrained('unsloth/Llama-3.2-1B')
model